# Anomaly Detection in Time Series with aeon

This notebook is currently under construction! Please check back later.

If you have any questions about the `aeon` anomaly detection module in the mean time, please ask us on Slack!

In [2]:
from aeon.registry import all_estimators

all_estimators("anomaly-detector", as_dataframe=True)

,name,estimator
0,DWT_MLEAD,<class 'aeon.anomaly_detection._dwt_mlead.DWT_...
1,KMeansAD,<class 'aeon.anomaly_detection._kmeans.KMeansAD'>
2,MERLIN,<class 'aeon.anomaly_detection._merlin.MERLIN'>
3,PyODAdapter,<class 'aeon.anomaly_detection._pyodadapter.Py...
4,STOMP,<class 'aeon.anomaly_detection._stomp.STOMP'>
5,STRAY,<class 'aeon.anomaly_detection._stray.STRAY'>
